In [ ]:
import kagglehub
import pandas as pd


In [ ]:
# path = kagglehub.dataset_download("stanislavlevendeev/haz-mat-signs")
path = "C:/Users/ewald/.cache/kagglehub/datasets/stanislavlevendeev/haz-mat-signs/versions/5"
print("Path to dataset files:", path)

In [ ]:
#show images in path/images
images_path = path + "/images"

import os
import matplotlib.pyplot as plt
import cv2

def show_images_from_path(path):
    """Loops through all image files in a given path and displays them using matplotlib."""
    valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif')
    image_files = [f for f in os.listdir(path) if f.lower().endswith(valid_extensions)]
    stop = 0
    for image_file in image_files:
        if stop <= 10:
            image_path = os.path.join(path, image_file)
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for correct color display
            
            plt.figure()
            plt.imshow(image)
            plt.title(image_file)
            plt.axis('off')
            plt.show()
            stop+=1


# Example usage
show_images_from_path(images_path)

In [ ]:
# show json file annotations coco format
annotations_path = path + "/hazmat_coco.json"
import json

def read_json(file_path):
    """Reads a JSON file and returns the data as a dictionary."""
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

# Example usage
json_data = read_json(annotations_path)
print(json_data)


    "categories": [
        {
            "id": 1,
            "name": "hazmat code"
        }
    ]

In [ ]:
json_data["categories"] = []
# add category to json file
new_category = {
    "id": 1,
    "name": "hazmat code"
}

json_data["categories"].append(new_category)


In [ ]:
json_data

In [ ]:
# make hazmat signs format
import torch
from torch.amp import autocast, GradScaler
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.transforms as T
import torchvision.transforms.functional as F
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.ops import MultiScaleRoIAlign
import os
import numpy as np
import matplotlib.pyplot as plt
import json
from PIL import Image, ImageDraw
import contextlib
import io
import time
from datetime import datetime
from tqdm import tqdm
from torchvision.transforms import ColorJitter, GaussianBlur
import random
import matplotlib.patches as patches

class HazmatDataset(Dataset):
    def __init__(self, data_dir, annotations_file, transforms=None):
        self.data_dir = data_dir
        self.transforms = transforms

        # Load annotations
        with open(annotations_file) as f:
            data = json.load(f)

        self.images = {img['id']: img for img in data['images']}
        self.annotations = data['annotations']

        # Create image_id to annotations mapping
        self.img_to_anns = {}
        for ann in self.annotations:
            img_id = ann['image_id']
            if img_id not in self.img_to_anns:
                self.img_to_anns[img_id] = []
            self.img_to_anns[img_id].append(ann)

        self.ids = list(self.images.keys())

    def __getitem__(self, idx):
        img_id = self.ids[idx]
        img_info = self.images[img_id]

        # Load image
        img_path = os.path.join(self.data_dir, 'images', img_info['file_name'])
        img = Image.open(img_path).convert('RGB')

        # Get annotations
        anns = self.img_to_anns.get(img_id, [])

        boxes = []
        labels = []
        areas = []
        iscrowd = []

        for ann in anns:
            bbox = ann['bbox']
            # Convert [x, y, w, h] to [x1, y1, x2, y2]
            x_min = bbox[0]
            y_min = bbox[1]
            x_max = bbox[0] + bbox[2]
            y_max = bbox[1] + bbox[3]

            # Filter out degenerate boxes
            if x_max > x_min and y_max > y_min:
                boxes.append([x_min, y_min, x_max, y_max])
                labels.append(ann['category_id'])
                areas.append(ann['area'])
                iscrowd.append(ann['iscrowd'])

        # Convert to tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        areas = torch.as_tensor(areas, dtype=torch.float32)
        iscrowd = torch.as_tensor(iscrowd, dtype=torch.int64)

        target = {
            'boxes': boxes,
            'labels': labels,
            'image_id': torch.tensor([img_id]),
            'area': areas,
            'iscrowd': iscrowd
        }

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.ids)

class RandomHorizontalFlip(object):
    def __init__(self, prob):
        self.prob = prob

    def __call__(self, image, target):
        if torch.rand(1) < self.prob:
            height, width = image.shape[-2:]
            image = F.hflip(image)
            # Flip bounding boxes
            bbox = target["boxes"]
            bbox[:, [0, 2]] = width - bbox[:, [2, 0]]  # Flip x-coordinates
            target["boxes"] = bbox
        return image, target

class ToTensor(object):
    def __call__(self, image, target):
        # Convert PIL image to tensor
        image = F.to_tensor(image)
        return image, target
    
def get_transform(train):
    transforms = []
    # Convert PIL image to tensor
    transforms.append(ToTensor())
    if train:
        # Add training augmentations here if needed
        transforms.append(RandomHorizontalFlip(0.5))
    return transforms

# make hazmat dataset from public dataset
test_dataset = HazmatDataset(
    data_dir=images_path,
    annotations_file=annotations_path,
    transforms=get_transform(train=False)
)


In [ ]:
from torch.utils.data import DataLoader
def collate_fn(batch):
    return tuple(zip(*batch))
workers = 1
# Create test data loader
test_loader = DataLoader(
    test_dataset,
    batch_size=2,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=workers,
    pin_memory=True
)

In [ ]:
# Initialize model
num_classes = 2  # hazmat code and background

# Create ResNet-101 backbone with FPN
backbone = resnet_fpn_backbone('resnet101', pretrained=True)

# Define anchor generator for FPN
anchor_generator = AnchorGenerator(
    sizes=((32,), (64,), (128,), (256,), (512,)),
    aspect_ratios=((0.5, 1.0, 2.0),) * 5
)

# Multi-scale RoI pooling for FPN
roi_pooler = MultiScaleRoIAlign(
    featmap_names=['0', '1', '2', '3', '4'],
    output_size=7,
    sampling_ratio=2
)

print("initializing model...")
# Initialize Faster R-CNN with ResNet-101-FPN
model = FasterRCNN(
    backbone=backbone,
    num_classes=num_classes,
    rpn_anchor_generator=anchor_generator,
    box_roi_pool=roi_pooler
)

device = torch.device('cpu')
print(f"Training model on {device}")

# Move model to device
model.to(device)

In [ ]:
def convert_to_coco_format(outputs, image_ids):
    coco_results = []
    for output, image_id in zip(outputs, image_ids):
        boxes = output['boxes'].cpu().numpy()
        scores = output['scores'].cpu().numpy()
        labels = output['labels'].cpu().numpy()
        
        for box, score, label in zip(boxes, scores, labels):
            coco_results.append({
                'image_id': image_id,
                'category_id': int(label),
                'bbox': [box[0], box[1], box[2] - box[0], box[3] - box[1]],
                'score': float(score)
            })
    return coco_results

# Validation Function
def validate(model, data_loader, coco_gt, device):
    model.eval()
    results = []

    # Add tqdm
    progress_bar = tqdm(data_loader, desc="Validation", leave=True)

    with torch.no_grad():
        for images, targets in progress_bar:
            images = list(image.to(device) for image in images)
            outputs = model(images)
            
            image_ids = [target['image_id'].item() for target in targets]
            coco_results = convert_to_coco_format(outputs, image_ids)
            results.extend(coco_results)

            # Update tqdm-bar
            progress_bar.set_postfix({"Processed": len(results)})

    if not results:
        print("No predictions generated. Skipping evaluation.")
        return [0.0] * 6  # Return dummy metrics for empty results

    # Suppress COCOeval output
    with contextlib.redirect_stdout(io.StringIO()):
        coco_dt = coco_gt.loadRes(results)
        coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
        coco_eval.evaluate()
        coco_eval.accumulate()
        coco_eval.summarize()

    return coco_eval.stats

In [ ]:
model_path = os.path.join('checkpoints/best_model.pth')
checkpoint = torch.load(model_path, map_location=device)

model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)

# Load ground truth annotations for test set
coco_test = COCO('data/data_faster_rcnn/test/annotations/instances_test.json')

# Evaluate on test set
test_metrics = validate(model, test_loader, coco_test, device)

# Print test metrics
print(f"Test Metrics - mAP: {test_metrics[0]:.4f}")
print(f"mAP@0.5: {test_metrics[1]:.4f}, mAP@0.75: {test_metrics[2]:.4f}")
print(f"mAP medium: {test_metrics[4]:.4f}, mAP large: {test_metrics[5]:.4f}")

In [ ]:
test_dataset.annotations